In [ ]:
#
%matplotlib inline
from IPython.display import clear_output
import matplotlib
import matplotlib_inline.backend_inline
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf", "retina")  # For export
import sys 
sys.path.append("../src/")
import seaborn as sns 
import altair as alt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from cycler import cycler
figure_path='../figures/SmartGridPaper/'
az.style.use(["science", 'arviz-doc'])
nice_fonts = {
        # Use LaTeX to write all text
        "font.family": "serif",
        # Always save as 'tight'
        "savefig.bbox" : "tight",
        "savefig.pad_inches" : 0.05,
        "ytick.right" : False,
        "font.serif" : "Times New Roman",
        "mathtext.fontset" : "dejavuserif",
        "axes.labelsize": 15,
        "font.size": 15,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 10,
        'legend.facecolor':'white',
        'legend.edgecolor': 'gray',
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        # Set line widths
        "axes.linewidth" : 0.5,
        "grid.linewidth" : 0.5,
        "lines.linewidth" : 1.,
        # Remove legend frame
        "legend.frameon" :False,
        # color-blind friendly cycle designed using https://colorcyclepicker.mpetroff.net/
        # see preview and check for colorblindness here https://coolors.co/107591-00c0bf-f69a48-fdcd49-8da798-a19368-525252-a6761d-7035b7-cf166e
        'axes.prop_cycle': cycler(color=['#107591','#00c0bf','#f69a48','#fdcd49','#8da798','#a19368','#525252','#a6761d','#7035b7','#cf166e'])      
}
matplotlib.rcParams.update(nice_fonts)
from utils.data_processing import add_time_features
import altair as alt
alt.themes.enable("opaque")
alt.data_transformers.disable_max_rows()


## Define metrics functions

In [ ]:
## Metrics functions
from sklearn.metrics import r2_score
from utils.significant_test import run_statistical_test
from utils.metrics import get_residual, get_mae, get_smape, get_mape, get_nbias, get_residual, get_mse, get_pointwise_metrics

def get_daily_metrics(pred:np.array, true:np.array):
    metrics=[]
    for i in range(len(true)):
        metric = get_pointwise_metrics(pred[i], true[i])
        metrics.append(pd.DataFrame.from_dict(metric, orient='index').T)
    return pd.concat(metrics)


# functions to get all metrics
def get_results_per_experiment(models, dataset, experiment_name, window_type, cross_valid=list(range(1,11))):
    file_name=f'{experiment_name}_{window_type}' if window_type is not None else experiment_name
    all_metrics=[]
    all_residual={}
    all_data=[]
    # [ 'TFT', 'NBEATS',  'NHiTS', 'RNN', 'MSTL', 'SeasonalNaive', 'AutoARIMA',   "RF", 'CATBOOST', 'LREGRESS']
    for encoder_type in models:
        metrics_per_model=[]
        residual_per_model=[]
        data_per_model=[]
        print(encoder_type)
        print()
        for cross in tqdm(cross_valid):
            if window_type=='train_test':
                result_path=f'../results/{file_name}/{encoder_type}/{cross}_processed_results.npy'
                #result_path_naive=f'../results/{file_name}/SeasonalNaive/{cross}_processed_results.npy'
            else:
                result_path=f'../results/{file_name}/{encoder_type}/{cross}_cross_validation_processed_results.npy'
                #result_path_naive=f'../results/{file_name}/SeasonalNaive/{cross}_cross_validation_processed_results.npy'
            results=np.load(result_path, allow_pickle=True).item()
            #results_naive=np.load(result_path_naive, allow_pickle=True).item()
            time_stamp=results['index']
            true=results['true']
            pred=results['pred']
            df = pd.DataFrame(time_stamp.flatten()[:len(true.flatten())])
            df.columns=['Date']
            df[f'{encoder_type}']=pred.flatten()
            df['true']=true.flatten()
            data_per_model.append(df)
            
            metrics=results['NetLoad_metrics']
            index=df.Date.dt.round("D").unique()
            
            residual=true-pred
            residual_per_model.append(residual.flatten())
            #pred_naive=results_naive['pred']
            #true_naive=results_naive['true']
            
            metrics['Fold']=cross
            metrics['Model']=encoder_type
            if 'train-time' in results.keys() :
                metrics['Train-time']=results['train-time'] 
            metrics['Test-time']=results['test-time'] if 'test-time' in results.keys() else results['test_time']
            metrics=metrics.set_index('timestamp')
            metrics=add_time_features(metrics, hemisphere = 'Northern')
            metrics=metrics.replace('CATBOOST', 'CAT').replace('AutoARIMA', 'ARIMA').replace('LREGRESS', 'L-REG').replace('SeasonalNaive', 'S-Naive').replace('MLPForecast', 'MLPF')
            metrics_per_model.append(metrics)
        all_residual[encoder_type]=residual_per_model
        metrics_per_model=pd.concat(metrics_per_model)
        all_metrics.append(metrics_per_model)
        all_data.append(pd.concat(data_per_model))

    metrics=pd.concat(all_metrics)
  
    if dataset=="UK":
        metrics['Dataset']='SPS-UK'
    else:
        metrics['Dataset']='MLVS-PT'
    all_data=pd.concat(all_data, axis=1)
    del all_metrics
    del df 
    del results
    #del results_naive
    del pred
    del true
    
    return metrics, all_residual, all_data

In [ ]:
forecast_len=6 
incremental_len = 2
min_train_len=12
seed=777
max_epochs=50                   
window_type='expanding'          
dataset="UK", # Put name of Dataset
experiment_name="UK-BestPractises" # Put name of Experiment

## Experiment 1: Metrics

#### Objectives: 

- To understand their suitability in accurately assessing net-load forecasting, emphasizing critical aspects such as peak values and negative net-load and how each metric rank the baselines models
- Metrics considered: NRMSE, MAE, SMAPE, MAPE, NBIAS

ad.

In [ ]:
models=['MSTL', 'SeasonalNaive',   "RF",  'LREGRESS', 'CATBOOST'] #,

def get_results_per_dataset(models, dataset, experiment_name, window_type=None):
    
    
    file_name=f'{experiment_name}_{window_type}' if window_type is not None else experiment_name
    all_metrics=[]
    all_residual={}
    for encoder_type in models:
        print(encoder_type)
        print()
        for cross in tqdm(range(1, 11)):
            result_path=f'../results/{file_name}/{encoder_type}/{cross}_cross_validation_processed_results.npy'
            results=np.load(result_path, allow_pickle=True).item()
            time_stamp=results['index'].flatten()
    
            metrics=results['NetLoad_metrics']
            df=pd.DataFrame(index=time_stamp)
            df['pred']=results['pred'].flatten()
            df['true']=results['true'].flatten()
            all_metrics.append(df)
            
    results = pd.concat(all_metrics)
    if dataset=="UK":
        results['Dataset']='SPS-UK'
    else:
        results['Dataset']='MLVS-PT'
    results['mae']=get_mae(results['true'].values, results['pred'].values)
    results['nbias']=get_nbias(results['true'].values, results['pred'].values)
    results['mse']=get_mse(results['true'].values, results['pred'].values)
    results['rmse']=results['mse']**0.5
    results['mape']=get_mape(results['true'].values, results['pred'].values)
    results['residual']=results['true'].values-results['pred'].values
    results['smape']=get_smape(results['true'].values, results['pred'].values)
    
    results['timestamp']=results.index
    results=results.set_index('timestamp')
    results=add_time_features(results, hemisphere = 'Northern')

    results['nrmse']=results.rmse/results.true.max()
    
    return results


## Best practises results analysis

In [ ]:
window_type='expanding'          
dataset="UK"
experiment_name="UK-BestPractises"
uk_results=get_results_per_dataset(['CATBOOST'], dataset, experiment_name, window_type)
uk_lowest_netload=np.where(uk_results['true']<0)[0]
uk_highest_netload=np.where(uk_results['true']>5)[0]

In [ ]:
pt_dataset="PT",
pt_experiment_name="PT-BestPractises"
pt_results=get_results_per_dataset(['CATBOOST'], pt_dataset, pt_experiment_name, window_type)
pt_lowest_netload=np.where(pt_results['true']<0)[0]
pt_highest_netload=np.where(pt_results['true']>90)[0]

In [ ]:
results_combined= pd.concat((pt_results, uk_results))

### Metrics with peaks and valley PT'dataset

In [ ]:
figure_path='../figures/MLPPaper/'

In [ ]:
!mkdir figure_path

In [ ]:
def plot_metrics_for_extreame_scenarios(metric_name, dataset, results, isPeak,  index):
    if isPeak:
        if dataset == "PT":
            highest_netload=np.where(results['true']>90)[0]
        else:
            highest_netload=np.where(results['true']>5)[0]
        idx=highest_netload[index]
    else:
        lowest_netload=np.where(results['true']<0)[0]
        idx=lowest_netload[index]
        print
    df=results.iloc[idx-48:idx+48]
    #df[metric_name]= df[metric_name].values/df[metric_name].max()
    fig, ax = plt.subplots(1, 1, figsize=(3.5, 2.5))
    true=df['true'].values
    pred=df['pred'].values
    #ax.step(np.arange(len(true)), true.astype(int), c="C2",  label='True')
    #ax.step(np.arange(len(true)), pred.astype(int),  c='C0', label='Pred')
    ax.plot(df['true'].values, ".", mec="C2", mfc="None", label='True')
    ax.plot(df['pred'].values,  '--',  c='C0', label='Pred')
    ax.fill_between(np.arange(len(true)), true, pred,  color='k', alpha=0.1, label='Forecast Error')
    ax.set_ylabel('Power (kW)')
    ax.legend(loc='upper left', fontsize=10)

    ax.set_xticks([i for i in range(0,108,12)])
    

    ax0 = ax.twinx()
    ax0.step(np.arange(len(df)), df[metric_name].values, c='C1', label=metric_name.upper())
    #ax0.plot(df[metric_name].values, c='C1', label=metric_name.upper())
    ax0.set_ylabel(metric_name.upper())
    ax0.legend(loc='upper right', fontsize=10)
    fig.savefig(f'{figure_path}{dataset}_{metric_name}_with_valley_and_peaks.pdf', dpi=480)


    fig=sns.jointplot(data=df, x="residual", y=metric_name, kind="reg", palette="tab10", height=2.8, color='C6')
    fig.set_axis_labels('residual', metric_name.upper(), fontsize=12)
    fig.figure.tight_layout()
    fig.savefig(f'{figure_path}{dataset}_{metric_name}_joint_with_valley_and_peaks.pdf', dpi=480)
    

In [ ]:
dataset = "PT"
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias']:
    plot_metrics_for_extreame_scenarios(metric_name, dataset, pt_results, False,  1)
    #plt.ylim(-5, 110)
    #plt.xlim(0, 100)

In [ ]:
dataset = "UK"
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias']:
    plot_metrics_for_extreame_scenarios(metric_name, dataset, uk_results, False,  50)
    #plt.ylim(-5, 6)
    #plt.xlim(0, 100)

## Residual vs metric joint plots

In [ ]:
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias', "residual"]:
    uk_results[metric_name]= uk_results[metric_name]/uk_results[metric_name].max()

    pt_results[metric_name]= pt_results[metric_name]/pt_results[metric_name].max()

results = pd.concat([uk_results, pt_results])

In [ ]:
fig=sns.jointplot(data=results, x="residual", y="mae", kind="scatter", palette="tab10", height=3.8,  hue='Dataset', )
fig.set_axis_labels('residual', 'MAE', fontsize=12)
fig.figure.tight_layout

fig.savefig(f'{figure_path}/residual_joint_mae.png', dpi=480)

In [ ]:
fig=sns.jointplot(data=results, x="residual", y="mape",  palette="tab10", height=3.8,hue='Dataset')
fig.set_axis_labels('residual', 'MAPE', fontsize=12)
fig.figure.tight_layout()
fig.savefig(f'{figure_path}/residual_joint_mape.png', dpi=480)

In [ ]:
fig=sns.jointplot(data=results, x="residual", y="smape", palette="tab10", height=3.8, hue='Dataset')
fig.set_axis_labels('residual', 'SMAPE', fontsize=12)
fig.figure.tight_layout()
fig.savefig(f'{figure_path}/residual_joint_smape.png', dpi=480)

In [ ]:
fig=sns.jointplot(data=results, x="residual", y="nrmse", palette="tab10",height=3.8,  hue='Dataset')
fig.set_axis_labels('residual', 'NRMSE', fontsize=12)
fig.figure.tight_layout()
fig.savefig(f'{figure_path}/residual_joint_nrmse.png', dpi=480)

In [ ]:
fig=sns.jointplot(data=results, x="residual", y="nbias", palette="tab10", height=3.8,  color='C6')
fig.set_axis_labels('residual', 'NBIAS', fontsize=12)
fig.figure.tight_layout()
fig.savefig(f'{figure_path}/residual_joint_nbias.png', dpi=480)

## Metrics across different models

In [ ]:
pt_all_metrics={}
pt_all_residual={}
pt_all_data={}
dataset='PT'
experiment_name="PT-BestPractises"
pt_installed_capacity=pt_results['true'].max()
for  window_type in ['expanding',  'train_test']:
    
    if window_type=='train_test':
        cross_valid=['test-two', 'test-one']
    else:
        cross_valid=list(range(1, 11))
    metrics, residual, data=get_results_per_experiment(models, dataset, experiment_name, window_type, cross_valid)
    metrics['window']=window_type
    pt_all_metrics[window_type]=metrics
    pt_all_residual[window_type]=residual
    pt_all_data[window_type]=data
    
pt_metrics=pd.concat(pt_all_metrics.values())
pt_metrics['nrmse']=pt_metrics['rmse']/pt_installed_capacity

In [ ]:
uk_all_metrics={}
uk_all_residual={}
uk_all_data={}
uk_dataset='UK'
experiment_name="UK-BestPractises"
uk_installed_capacity=uk_results['true'].max()
for  window_type in ['expanding',  'train_test']:
    
    if window_type=='train_test':
        cross_valid=['test-two', 'test-one']
    else:
        cross_valid=list(range(1, 11))
    metrics, residual, data=get_results_per_experiment(models, uk_dataset, experiment_name, window_type, cross_valid)
    metrics['window']=window_type
    uk_all_metrics[window_type]=metrics
    uk_all_residual[window_type]=residual
    uk_all_data[window_type]=data
    
uk_metrics=pd.concat(uk_all_metrics.values())
uk_metrics['nrmse']=uk_metrics['rmse']/uk_installed_capacity

In [ ]:
met=['nrmse', 'mae', 'mape', 'smape', 'nbias']
metrics = pd.concat([uk_metrics, pt_metrics])
res=metrics[metrics['window']=='expanding'].groupby(['Dataset','Model'])[met].median().sort_values('mae').round(3)
#res=metrics[metrics['window']=='expanding'].groupby('Model')[met].std().sort_values('nrmse').round(2)
table = res.to_latex(index=True, formatters={"name": str.upper},
                  float_format="{:.3f}".format)
# Print the LaTeX table
print(res)

### Analyse how each metric rank different models across seasons

In [ ]:
az.style.use(["science", 'arviz-doc'])
def per_metric_plot(data, metric='nrmse'):
    fig=sns.catplot(data=data, 
                    x='Model', 
                    y=metric, 
                    estimator='median',
                    hue='Season', 
                    hue_order=['Autumn', 'Winter', 'Spring', 'Summer'],
                    sharex=False, margin_titles=True,
                    height=3.28, aspect=1.0,
                    #order=['CAT',  'LSTM',  'NHiTS', 'NBEATS', "MLPFPQ"],
                    #palette={"MLVS-PT": "C1", "SPS-UK": "C2"},
                    order=[ 'CAT', 'L-REG',  'MSTL', 'S-Naive'],
                     kind='point',
                    legend='auto', legend_out=False)
    return fig

In [ ]:
dataset='PT'
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias']:
    fig= per_metric_plot(pt_metrics[pt_metrics['window']=='expanding'].reset_index(), metric=metric_name)
    fig.set_axis_labels( 'Model', metric_name.upper(), fontsize=12)
    fig.figure.tight_layout() 
    fig.set_titles(row_template="{row_name} Season")
    plt.legend(loc='lower right', fontsize=10)
    plt.xticks(rotation=90);
    if metric_name=='mae':
        fig.set(ylim=(1, 15))
    elif metric_name=='mape':
        fig.set(ylim=(0, 0.4))
    elif metric_name=='smape':
        fig.set(ylim=(0, 0.4))
    elif metric_name=='nrmse':
        fig.set(ylim=(0, 0.2))
    elif metric_name=='nbias':
        plt.axhline(y=2, color='black', linestyle='--', alpha=0.21)
        plt.axhline(y=-2, color='black', linestyle='--', alpha=0.21)
        fig.set(ylim=(-7.5, 8.0))
    fig.savefig(f'{figure_path}/{dataset}_Expanding_{metric_name}.pdf', dpi=480)

In [ ]:
dataset='UK'
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias']:
    fig= per_metric_plot(uk_metrics[uk_metrics['window']=='expanding'].reset_index(), metric=metric_name)
    fig.set_axis_labels( 'Model', metric_name.upper(), fontsize=12)
    fig.figure.tight_layout() 
    fig.set_titles(row_template="{row_name} Season")
    plt.legend(loc='lower right', fontsize=10)
    plt.xticks(rotation=90);
    if metric_name=='mae':
        fig.set(ylim=(0, 2))
    elif metric_name=='mape':
        fig.set(ylim=(0, 3))
        #pass
    elif metric_name=='smape':
        fig.set(ylim=(0, 0.8))
    elif metric_name=='nrmse':
        fig.set(ylim=(0, 0.3))
    elif metric_name=='nbias':
        plt.axhline(y=2, color='black', linestyle='--', alpha=0.21)
        plt.axhline(y=-2, color='black', linestyle='--', alpha=0.21)
        fig.set(ylim=(-25, 15))
    fig.savefig(f'{figure_path}/{dataset}_Expanding_{metric_name}.pdf', dpi=480)

## Statistical test

In [ ]:
def get_net_load(data):
    net_load=data.true.iloc[:, 0]
    data=data[[  'MSTL','SeasonalNaive',  'RF',  'CATBOOST', 'LREGRESS']]
    data=data.rename(columns={'SeasonalNaive':'S-NAIVE',  'LREGRESS':'L-REG', 'CATBOOST':'CAT',  'AutoARIMA':'ARIMA'})
    data['NetLoad']=net_load.values
    # Transforming indices to datetime format
    data.index = pd.to_datetime(data.index)
    data.head()
    real_netload = data.loc[:, ['NetLoad']]
    del data['NetLoad']
    return data, real_netload

In [ ]:
data=uk_all_data['expanding']
data, real_netload=get_net_load(data)

In [ ]:
real_netload.shape

In [ ]:
dataset='UK'
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_value=run_statistical_test(ax, real_netload, data, norm=2, test='GW')
#ax.set_title('')
fig.savefig(f'{figure_path}/{dataset}_GW_test_model.pdf', dpi=480)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_values=p_value=run_statistical_test(ax, real_netload, data, norm=2, test='DM')
#ax.set_title('')
fig.savefig(f'{figure_path}/{dataset}_DM_test_model.pdf', dpi=480)

Note: P-value close to zero represent cases where the forecast in the x-axis is significant more accurate than the forecast in the y-axis

In [ ]:
data=all_data['train_test']
data, real_netload=get_net_load(data)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_values=p_value=run_statistical_test(ax, real_netload, data, norm=1, test='DM')
#fig.savefig(f'{figure_path}/DM_test_model_test_2.pdf', dpi=480)

## Metrics with different data spiliting

In [ ]:
def per_dtype_plot(data, metric='nrmse'):
    fig=sns.catplot(data=data, 
                    x='Model', 
                    y=metric, 
                    hue='D-type', 
                    estimator='median',
                    #hue_order=['Autumn', 'Winter', 'Spring', 'Summer'],
                    sharex=False, margin_titles=True,
                    height=3.5, aspect=1.0,
                    #order=['CAT',  'LSTM',  'NHiTS', 'NBEATS', "MLPFPQ"],
                    #palette={"MLVS-PT": "C1", "SPS-UK": "C2"},
                    order=[ 'CAT', 'RF',  'MSTL', 'S-Naive'],
                     kind='point',
                   legend='auto', legend_out=False)
    return fig

In [ ]:
test_one=metrics[metrics['Fold']=='test-one']
#test_one=test_one[test_one.Season=='Winter']
test_one['D-type']='Train-test-1'
test_two=metrics[metrics['Fold']=='test-two']
#test_two=test_two[test_two.Season=='Summer']
test_two['D-type']='Train-test-2'
window=metrics[metrics['window']=='expanding']
window['D-type']='Expanding'
data_splitting=pd.concat([test_one, test_two, window])

In [ ]:
data_splitting.Dataset.unique()

In [ ]:
uk_data_splitting=data_spiliting[data_spiliting.Dataset=='SPS-UK']
pt_data_splitting=data_spiliting[data_spiliting.Dataset=='MLVS-PT']

In [ ]:
def plot_dtype(data_splitting, dataset, metric_name):
    data_splitting=data_splitting[data_splitting.Model.isin(['CAT', 'MSTL', 'RF', 'S-Naive'])]
    fig= per_dtype_plot(data_splitting.reset_index(), metric=metric_name)
    fig.set_axis_labels( 'Model', metric_name.upper(), fontsize=12)
    fig.figure.tight_layout() 
    fig.set_titles(row_template="{row_name} Season")
    plt.legend(loc='lower right', fontsize=10)
    plt.xticks(rotation=90);
    #fig.set(ylim=(0, 10))
    fig.savefig(f'{figure_path}/{dataset}_D-type_MAE.pdf', dpi=480)

In [ ]:
pt_metrics.groupby(['Model'])[met].max()

In [ ]:
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias']:
    plot_dtype(pt_data_splitting, 'PT', metric_name)

In [ ]:
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias']:
    plot_dtype(uk_data_spiliting, 'UK', metric_name)

In [ ]:
res=data_spiliting.groupby(['Dataset','D-type','Model'])[met].median().round(3)
#res=metrics[metrics['window']=='expanding'].groupby('Model')[met].std().sort_values('nrmse').round(2)
table = res.to_latex(index=True, formatters={"name": str.upper},
                  float_format="{:.3f}".format)
# Print the LaTeX table
#print(table)
res

In [ ]:
for metric_name in ['mae','mape', 'smape', 'nrmse', 'nbias']:
    plot_dtype(uk_data_spiliting, 'UK', metric_name)

In [ ]:
fig= per_dtype_plot(data_spiliting.reset_index(), metric='mape')
fig.set_axis_labels( 'Model', 'MAPE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper right', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 0.6))

In [ ]:
fig= per_dtype_plot(data_spiliting.reset_index(), metric='smape')
fig.set_axis_labels( 'Model', 'SMAPE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper right', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 0.6))

In [ ]:
fig= per_dtype_plot(data_spiliting.reset_index(), metric='nbias')
fig.set_axis_labels( 'Model', 'NBIAS', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper left', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(-10, 10))

In [ ]:
data_spiliting.groupby(['D-type', 'Model']).median().sort_values('mae').round(2)[met]

In [ ]:
data=metrics[metrics['window']=='expanding']
data=data[data['Model'].isin([ 'S-Naive','L-REG', 'MSTL', 'RF',  'NHiTS', 'RNN',  'CAT', 'NBEATS'])]

In [ ]:
altair_plot_avg(short_long_metrics, metric='mae', column='Context', dataset='PT',  across='Horizon', limit=[0,10])

## Model-design

In [ ]:

experiment_type='mlpf-model-design'
exp_name="PT-Benchmark"
dataset='pt_dataset'
model_design_metrics=[]
data_design=[]
for emb_type in [ "None", 'PosEmb', 'RotaryEmb']: #'PosEmb', 'RotaryEmb']:
    for comb_type in ['attn-comb', 'weighted-comb', 'addition-comb']:
        experiment_name=f'{exp_name}_{dataset}_{experiment_type}_{emb_type}_{comb_type}' 
        metrics, res, data=get_results_per_experiment(['MLPForecast'], 
                                                      dataset, experiment_name, 
                                                      None, cross_valid=list(range(1,11)))
        metrics['Comb-Type']=comb_type
        metrics['Emb-Type']=emb_type
        data['Comb-Type']=comb_type
        data['Emb-Type']=emb_type                                                                                  
        data_design.append(data)
        model_design_metrics.append(metrics)
        
model_design_metrics_pt = pd.concat(model_design_metrics)
model_design_metrics_pt=model_design_metrics_pt.replace('addition-comb', 'ADD').replace('attn-comb', 'ATTN').replace('weighted-comb', 'WTA')
model_design_metrics_pt=model_design_metrics_pt.replace('PosEmb', 'PoE').replace('RotaryEmb', 'RoPE').replace('CombinedEmb', 'Comb')
model_design_metrics_pt['Dataset']='MLVS-PT'
data_design_pt = pd.concat(data_design)                                                      
clear_output()

In [ ]:
model_design_metrics_pt.groupby(['Comb-Type', 'Emb-Type'])[['mae', 'nrmse']].mean().round(2)

In [ ]:
exp_name="PT-Benchmark"
dataset='uk_dataset'
model_design_metrics_uk=[]
data_design_uk=[]
for emb_type in [ "None", 'PosEmb', 'RotaryEmb' ]: #'PosEmb', 'RotaryEmb', 'CombinedEmb']:
    for comb_type in ['attn-comb', 'weighted-comb', 'addition-comb']:
        experiment_name=f'{exp_name}_{dataset}_{experiment_type}_{emb_type}_{comb_type}' 
        metrics, res, data=get_results_per_experiment(['MLPForecast'], 
                                                      dataset, experiment_name, 
                                                      None, cross_valid=list(range(1,11)))
        metrics['Comb-Type']=comb_type
        metrics['Emb-Type']=emb_type
        data['Comb-Type']=comb_type
        data['Emb-Type']=emb_type                                                                                  
        data_design_uk.append(data)
        model_design_metrics_uk.append(metrics)
        
model_design_metrics_uk = pd.concat(model_design_metrics_uk)
model_design_metrics_uk=model_design_metrics_uk.replace('addition-comb', 'ADD').replace('attn-comb', 'ATTN').replace('weighted-comb', 'WTA')
model_design_metrics_uk=model_design_metrics_uk.replace('PosEmb', 'PoE').replace('RotaryEmb', 'RoPE').replace('CombinedEmb', 'Comb')
model_design_metrics_uk['Dataset']='SPS-UK'
data_design_uk = pd.concat(data_design_uk)                                                      
clear_output()

In [ ]:
model_design_metrics_uk.groupby(['Comb-Type', 'Emb-Type'])[['mae', 'nrmse']].mean().round(2)

In [ ]:
def model_dsign_statistical_test_comb(ax, data, emb_type, test_type='DM', norm=1):
    data = data[data['Emb-Type'].isin([emb_type])]
    df = data.reset_index().groupby(['Date', 'Comb-Type'], as_index=False).mean()
    forecast = df.pivot(index='Date', columns='Comb-Type', values='MLPForecast')
    
    forecast=forecast.rename(columns={'addition-comb':'ADD', 'attn-comb':'ATTN', 'weighted-comb':'WTA'})
    true = df.pivot(index='Date', columns='Comb-Type', values='true')[['attn-comb']]
    #forecast=forecast.set_index('Date')
    #true=true.set_index('Date')
    true=true.rename(columns={'weighted-comb':'true'})
    p_value=run_statistical_test(ax,true, forecast, norm=norm, test=test_type)
    return p_value


def model_dsign_statistical_test_emb(ax, data, comb_type, test_type='DM', norm=1):
    data = data[data['Comb-Type'].isin([comb_type])]
    df = data.reset_index().groupby(['Date', 'Emb-Type'], as_index=False).mean()
    forecast = df.pivot(index='Date', columns='Emb-Type', values='MLPForecast')[["None", 'PosEmb', 'RotaryEmb']]
    forecast=forecast.rename(columns={'PosEmb':'PoE', 'RotaryEmb':'RoPE', 'CombinedEmb':'Comb'})
    true = df.pivot(index='Date', columns='Emb-Type', values='true')[['None']]
    #forecast=forecast.set_index('Date')
    #true=true.set_index('Date')
    true=true.rename(columns={'None':'true'})
    p_value=run_statistical_test(ax,true, forecast, norm=norm, test=test_type)
    return p_value
    
    


In [ ]:
for emb_type in ["None", 'PosEmb', 'RotaryEmb']:
    print(emb_type)
    fig, ax = plt.subplots(1,1, figsize=(3.5,3))
    pvalue=model_dsign_statistical_test_comb(ax, data_design_pt, emb_type=emb_type, test_type='DM', norm=1)
    print(pvalue)
    fig.savefig(f'{figure_path}/PT_{emb_type}_test_model.pdf', dpi=480)

In [ ]:
for emb_type in ["None", 'PosEmb', 'RotaryEmb']:
    print(emb_type)
    fig, ax = plt.subplots(1,1, figsize=(3.5,3))
    pvalue=model_dsign_statistical_test_comb(ax, data_design_uk, emb_type=emb_type, test_type='DM', norm=1)
    print(pvalue)
    fig.savefig(f'{figure_path}/UK_{emb_type}_DM_test_model.pdf', dpi=480)

In [ ]:
for comb_type in [ 'addition-comb']:
    print(comb_type)
    fig, ax = plt.subplots(1,1, figsize=(3.5,3))
    pvalue=model_dsign_statistical_test_emb(ax, data_design_pt, comb_type=comb_type, test_type='DM', norm=1)
    print(pvalue)
    fig.savefig(f'{figure_path}/PT_{comb_type}_DM_test_model.pdf', dpi=480)

In [ ]:
for comb_type in [ 'addition-comb', 'attn-comb', 'weighted-comb',]:
    print(comb_type)
    fig, ax = plt.subplots(1,1, figsize=(3.5,3))
    pvalue=model_dsign_statistical_test_emb(ax, data_design_uk, comb_type=comb_type, test_type='DM', norm=1)
    print(pvalue)
    fig.savefig(f'{figure_path}/UK_{comb_type}_DM_test_model.pdf', dpi=480)
   

In [ ]:
model_design_metrics=pd.concat([model_design_metrics_pt, model_design_metrics_uk])

In [ ]:
def altair_plot(data, metric='mae', column='Comb-Type', dataset='PT',  across='Emb-Type', domain=[0., 6]):
    #data=data[data['Emb-Type']=='None']
    #data=data.replace('addition-comb', 'ADD').replace('attn-comb', 'ATTN').replace('weighted-comb', 'WTA')
    line = alt.Chart(data).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    y = alt.Y(f'mean({metric})', scale=alt.Scale(domain=domain), title=metric.upper()),
    x = alt.X(f'{column}:N', axis=alt.Axis( title='')),
    color=alt.Color('Season', scale=alt.Scale(scheme='tableau20')),
    column=alt.Column(f'{across}:N', title=''),
    
    ).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
        ).configure_view(
            strokeOpacity=0
        ).properties(width=120,
                        height=150
        )
    line.save(f'{figure_path}/{dataset}_{column}_{metric}.pdf', ppi=480)
    return line


def altair_plot_avg(data, metric='mae', column='Comb-Type', dataset='PT',  across='Emb-Type', domain=[0., 6]):
    #data=data[data['Emb-Type']=='None']
    #data=data.replace('addition-comb', 'ADD').replace('attn-comb', 'ATTN').replace('weighted-comb', 'WTA')
    line = alt.Chart(data).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    y = alt.Y(f'mean({metric})', scale=alt.Scale(domain=domain), title=metric.upper()),
    x = alt.X(f'{column}:N', axis=alt.Axis( title='')),
    color=alt.Color(f'{across}:N', scale=alt.Scale(scheme='tableau20')),
    #column=alt.Column(f'{across}:N', title=''),
    
    ).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
        ).configure_view(
            strokeOpacity=0
        ).properties(width=120,
                        height=150
        )
    line.save(f'{figure_path}/{dataset}_{column}_{metric}_overall.pdf', ppi=480)
    return line




In [ ]:
alt.themes.enable("opaque")
alt.data_transformers.disable_max_rows()
altair_plot(model_design_metrics_pt, metric='mae', column='Comb-Type')

In [ ]:
altair_plot(model_design_metrics_uk, dataset='UK',  metric='mae', column='Comb-Type', domain=[0,0.6])

In [ ]:
altair_plot_avg(model_design_metrics_pt, metric='mae', column='Comb-Type')

In [ ]:
altair_plot_avg(model_design_metrics_uk, dataset='UK',  metric='mae', column='Comb-Type', domain=[0,0.6])

In [ ]:
altair_plot(model_design_metrics_pt, metric='mae', column='Emb-Type', across='Comb-Type')

In [ ]:
altair_plot(model_design_metrics_uk, dataset='UK', metric='mae', column='Emb-Type', across='Comb-Type', domain=[0,0.6])

In [ ]:
altair_plot_avg(model_design_metrics_pt, metric='mae', column='Emb-Type', across='Comb-Type', domain=[3, 4])

In [ ]:
altair_plot_avg(model_design_metrics_uk, dataset='UK', metric='mae', column='Emb-Type', across='Comb-Type', domain=[0,0.6])

In [ ]:
metrics_pt.columns

## Computation Metrics

## Benchmark

In [ ]:
forecast_len=6 
incremental_len = 2
min_train_len=12
seed=777
max_epochs=100    
n_splits=10
window_type='expanding'        
dataset="PT",
experiment_name="PT-Benchmark"

In [ ]:
models=['MLPForecast', 'NBEATS', 'NHiTS', 'LSTM', 'MSTL' ,'CATBOOST', 'SeasonalNaive', 'RF', 'LREGRESS', 'PatchTST', 'TimesNet', 'FEDformer']

In [ ]:
cross_valid=list(range(1, 11))
metrics_pt, residual_pt, data_pt=get_results_per_experiment(models, dataset, experiment_name, window_type, cross_valid)
clear_output()

In [ ]:
dataset="UK",
experiment_name="UK-Benchmark"
metrics_uk, residual_uk, data_uk=get_results_per_experiment(models, dataset, experiment_name, window_type, cross_valid)
clear_output()

In [ ]:
metrics_pt['Dataset']='MLVS-PT'
metrics_uk['Dataset']='SPS-UK'

In [ ]:
metrics=pd.concat([metrics_pt, metrics_uk])

In [ ]:
metrics_uk['Test-time']=(metrics_uk['Test-time']/60)
metrics_pt['Test-time']=(metrics_pt['Test-time']/60)
metrics['Test-time']=metrics['Test-time']/60


In [ ]:
metrics_uk['Train-time']=(metrics_uk['Train-time'])/60
metrics_pt['Train-time']=(metrics_pt['Train-time'])/60

## Inference speed

In [ ]:
alt.themes.enable("opaque")
alt.data_transformers.disable_max_rows()
chart=alt.Chart(metrics_uk).mark_bar().encode(
    x=alt.X('Model', title=None, sort=[ 'S-Naive', 'MSTL',  'MLPF',  'L-REG',  'RF', 'CAT', 'PatchTST', 'NHiTS', 'NBEATS', 'FEDformer' ,  'TimesNet',  'LSTM']),
    y=alt.Y('mean(Test-time):Q', scale=alt.Scale(domain=[0.001, 0.8]), title="Inference time (minutes)"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20'))

).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).resolve_scale(x='independent'
).properties(width=150,
                height=180
)
chart.save(f'{figure_path}/UK_inference_speed.pdf', ppi=480)
chart

In [ ]:
chart=alt.Chart(metrics_pt).mark_bar().encode(
    x=alt.X('Model', title=None, sort=[ 'S-Naive', 'MSTL',  'MLPF',  'L-REG',  'RF', 'CAT', 'NBEATS', 'NHiTS', 'LSTM', 'PatchTST',  'FEDformer' ,  'TimesNet']),
    y=alt.Y('mean(Test-time):Q', scale=alt.Scale(domain=[0.0, 0.8]), title="Inference time (minutes)"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20'))

).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).resolve_scale(x='independent'
).properties(width=150,
                height=180
)
chart.save(f'{figure_path}/PT_inference_speed.pdf', ppi=480)
chart

In [ ]:
df=metrics_pt.groupby('Model')[['Test-time', 'Train-time']].mean()

In [ ]:
df['Test-time']=df['Test-time']*60

In [ ]:
df.reset_index().sort_values('Test-time')

## Train time

In [ ]:
chart=alt.Chart(metrics_uk).mark_bar().encode(
    x=alt.X('Model', title=None, sort=[ 'L-REG','S-Naive', 'MSTL', 'NHiTS', 'NBEATS', 'MLPF',  'RF', 'CAT', 'LSTM', 'TimesNet',   'PatchTST',  'FEDformer' ]),
    y=alt.Y('mean(Train-time):Q', scale=alt.Scale(type='log'), title="Train time (minutes)"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20'))

).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).resolve_scale(x='independent'
).properties(width=150,
                height=180
)
chart.save(f'{figure_path}/UK_train_speed.pdf', ppi=480)
chart

In [ ]:
chart=alt.Chart(metrics_uk).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    x=alt.X('Fold', title=None),
    y=alt.Y('mean(Train-time):Q', scale=alt.Scale(domain=[0,10]), title="Train time (minutes)").scale(type="log"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20'))
   
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).properties(width=150,
                height=180
)
chart.save(f'{figure_path}/UK_train_speed_folds_speed.pdf', ppi=480)
chart

In [ ]:
chart=alt.Chart(metrics_pt).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    x=alt.X('Fold', title=None),
    y=alt.Y('mean(Train-time):Q', scale=alt.Scale(domain=[0,10]), title="Train time (minutes)"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20'))
   
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).properties(width=150,
                height=180
)
chart.save(f'{figure_path}/PT_train_speed_folds_speed.pdf', ppi=480)
chart

## Training time vs inference

In [ ]:

chart = alt.Chart(metrics_uk).mark_point(size=60).encode(
    y = alt.Y('mean(nrmse)', scale=alt.Scale(domain=[0.01, 0.3]), title='NMRSE'),
    x = alt.X('mean(Test-time)',scale=alt.Scale(domain=[0, 0.8]), axis=alt.Axis( title='Inference time \n (minutes)')),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20')),
    shape=alt.Shape('Model')
    
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).properties(width=180,
                height=200
)
chart.save(f'{figure_path}/UK_test_speed_nrmse.pdf', ppi=480)
chart

In [ ]:
chart = alt.Chart(metrics_pt).mark_point(size=60).encode(
    y = alt.Y('mean(nrmse)', scale=alt.Scale(domain=[0.0, 0.2]), title='NRMSE'),
    x = alt.X('mean(Test-time)',scale=alt.Scale(domain=[0.0, 0.8]), axis=alt.Axis( title='Inference time \n (minutes)')),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20')),
    shape=alt.Shape('Model')
    
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).properties(width=180,
                height=200
)
chart.save(f'{figure_path}/PT_test_speed_mae.pdf', ppi=480)
chart

In [ ]:
chart = alt.Chart(metrics_pt).mark_point(size=60).encode(
    y = alt.Y('mean(mae)', scale=alt.Scale(domain=[0.0, 12]), title='MAE'),
    x = alt.X('mean(Test-time)',scale=alt.Scale(domain=[0.0, 0.8]), axis=alt.Axis( title='Inference time \n (minutes)')),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20')),
    shape=alt.Shape('Model')
    
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).properties(width=180,
                height=200
)
chart.save(f'{figure_path}/PT_test_speed_nrmse.pdf', ppi=480)
chart

In [ ]:
chart = alt.Chart(metrics_uk).mark_point(size=60).encode(
    y = alt.Y('median(mae)', scale=alt.Scale(domain=[0.0, 2]), title='MAE'),
    x = alt.X('median(Test-time)',scale=alt.Scale(domain=[0.01, 0.8]), axis=alt.Axis( title='Inference time \n (minutes)')),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20')),
    shape=alt.Shape('Model')
    
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).properties(width=180,
                height=200
)
chart.save(f'{figure_path}/UK_test_speed_mae.pdf', ppi=480)
chart

In [ ]:
def print_results(pd_metrics):
     print(f"{pd_metrics['nrmse'].mean():.{2}f} $\pm$ {pd_metrics['nrmse'].std():.{2}f} & \
            {pd_metrics['mae'].mean():.{2}f} $\pm$ {pd_metrics['mae'].std():.{2}f} & \
            {pd_metrics['nbias'].mean():.{2}f} $\pm$ {pd_metrics['nbias'].std():.{2}f}  \\")
def formart_results(metrics_pt):
    
    for model_name in metrics_pt.sort_values('mae').Model.unique():
        pd_metrics=metrics_pt[metrics_pt.Model.isin([model_name])]
        print(model_name)
        print_results(pd_metrics)

In [ ]:
formart_results(metrics_uk)

In [ ]:
az.style.use(["science",  "arviz-doc", 'tableau-colorblind10'])

In [ ]:
fig=sns.catplot(data=metrics_uk, 
                x='Model', 
                y='nrmse', 
                #hue='Season', 
                order=['MLPF', 'NBEATS', 'PatchTST', 'TimesNet', 'FEDformer', 'NHiTS', 'LSTM', 'CAT', 'RF','L-REG', 'MSTL',  'S-Naive'],
                sharex=False, margin_titles=True,
                height=2.8, aspect=1.25, legend=False,
                estimator='mean', errorbar=('ci', 95),
                color='C2',
                #palette={"Summer": "C2", "Spring": "C1", "Autumn": "C3", "Winter": "C0"},
                 kind='box',showfliers=False, linecolor='k')
fig.set_axis_labels('', 'NRMSE', fontsize=10)
fig.figure.tight_layout() 
fig.set(ylim=(0, 0.5))
plt.xticks(rotation=90);
fig.savefig(f'{figure_path}/UK_bencmark_nrmse.pdf', dpi=480)

In [ ]:
fig=sns.catplot(data=metrics_uk, 
                x='Model', 
                y='mae', 
                #hue='Season', 
                order=['MLPF', 'NBEATS', 'PatchTST', 'TimesNet', 'FEDformer', 'NHiTS', 'LSTM', 'CAT', 'RF','L-REG', 'MSTL',  'S-Naive'],
                sharex=False, margin_titles=True,
                height=2.8, aspect=1.25, legend=False,
                estimator='mean', errorbar=('ci', 95),
                color='C2',
                #palette={"Summer": "C2", "Spring": "C1", "Autumn": "C3", "Winter": "C0"},
                 kind='box',showfliers=False, linecolor='k')
fig.set_axis_labels('', 'MAE(MW)', fontsize=10)
fig.figure.tight_layout() 
fig.set(ylim=(0, 2))
plt.xticks(rotation=90);
fig.savefig(f'{figure_path}/UK_bencmark_MAE.pdf', dpi=480)

In [ ]:
fig=sns.catplot(data=metrics_pt, 
                x='Model', 
                y='nrmse', 
                #hue='Season', 
                order=['MLPF', 'NBEATS',  'TimesNet', 'FEDformer', 'NHiTS', 'CAT', 'RF', 'LSTM','PatchTST','L-REG', 'MSTL',  'S-Naive'],
                sharex=False, margin_titles=True,
                height=2.8, aspect=1.25, legend=False,
                estimator='mean', errorbar=('ci', 95),
                color='C2',
                #palette={"Summer": "C2", "Spring": "C1", "Autumn": "C3", "Winter": "C0"},
                 kind='box',showfliers=False, linecolor='k')
fig.set_axis_labels('', 'NRMSE', fontsize=10)
fig.figure.tight_layout() 
fig.set(ylim=(0, 0.2))
plt.xticks(rotation=90);
fig.savefig(f'{figure_path}/PT_bencmark_nrmse.pdf', dpi=480)

In [ ]:
fig=sns.catplot(data=metrics_pt, 
                x='Model', 
                y='mae', 
                #hue='Season', 
                order=['MLPF', 'NBEATS',  'TimesNet', 'FEDformer', 'NHiTS', 'CAT', 'RF', 'LSTM','PatchTST','L-REG', 'MSTL',  'S-Naive'],
                sharex=False, margin_titles=True,
                height=2.8, aspect=1.25, legend=False,
                estimator='mean', errorbar=('ci', 95),
                color='C2',
                #palette={"Summer": "C2", "Spring": "C1", "Autumn": "C3", "Winter": "C0"},
                 kind='box',showfliers=False, linecolor='k')
fig.set_axis_labels('', 'MAE (kW)', fontsize=10)
fig.figure.tight_layout() 
fig.set(ylim=(0, 10))
plt.xticks(rotation=90);
fig.savefig(f'{figure_path}/PT_bencmark_mae.pdf', dpi=480)

In [ ]:
#res=metrics.groupby(['Dataset','Model'])['nrmse', 'mae',  'nbias'].median().round(3)
res=metrics_uk.groupby(['Dataset','Model'])[['nrmse', 'mae',  'nbias']].median().sort_values('nrmse').round(2)
table = res.to_latex(index=True, formatters={"name": str.upper},
                  float_format="{:.2f}".format)
# Print the LaTeX table
print(table)

#### statistical test

In [ ]:
def get_net_load(data):
    net_load=data.true.iloc[:, 0]
    data=data[models]
    data=data.rename(columns={'SeasonalNaive':'S-NAIVE',  'LREGRESS':'L-REG', 'CATBOOST':'CAT',  'MLPForecast':'MLPF'})
    data['NetLoad']=net_load.values
    # Transforming indices to datetime format
    data.index = data_pt.Date.iloc[:, 0]
    data.head()
    real_netload = data.loc[:, ['NetLoad']]
    del data['NetLoad']
    return data, real_netload

In [ ]:
pt_data, pt_netload=get_net_load(data_pt)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_value=run_statistical_test(ax, pt_netload, pt_data, norm=1, test='DM', sample_per_period=48)
#ax.set_title('')
fig.savefig(f'{figure_path}/PT_DM_test_model.pdf', dpi=480)

In [ ]:
uk_data, uk_netload=get_net_load(data_uk)
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_value=run_statistical_test(ax, uk_netload, uk_data, norm=1, test='DM', sample_per_period=48)
#ax.set_title('')
fig.savefig(f'{figure_path}/UK_DM_test_model.pdf', dpi=480)

## Error analysis

In [ ]:
from palettable.cmocean.diverging import Curl_20
from utils.data_processing import add_time_features
def prepareheatmap_df(data, hemisphere = 'Northern'):
    data = add_time_features(data, hemisphere)
    data['date']=data.index.date
    data["Time"] = data.index.time
    return data

def create_heatmap(ax, df, value, index='Time', column='date', cmap=Curl_20.get_mpl_colormap(), label='Net-Load (kW)', vmin=5, vmax=-5):
    data_df = df.copy()
    get_first = lambda x: x.iloc[0]
    # Pivot dates and times to create a two dimensional representation
    hm = data_df.pivot_table(index=index, columns=column, values=value, aggfunc=np.mean, dropna=True)
    sns.heatmap(hm, ax=ax, cbar_kws={'label': label}, cmap=cmap, vmin=vmin, vmax=vmax)
    return hm

def create_nbias(predictions, ground_truth):
    num =  predictions.values - ground_truth.values 
    denom = ground_truth.values + predictions.values
    nbias = pd.DataFrame(np.divide(num, denom))
    nbias.index=predictions.index
    nbias.index.name='timestamp'
    nbias.columns=predictions.columns
    return nbias

def create_res(predictions, ground_truth):
    
    
    res = pd.DataFrame(ground_truth.values - predictions.values)
    res.index=predictions.index
    res.index.name='timestamp'
    res.columns=predictions.columns
    return res

def plot_distribution(ax, df, index_col='HOUR', val_col='WindSpeed', hue_col=None):
    spivot = pd.pivot_table(df, index=index_col, values=val_col, columns=hue_col,  aggfunc=np.mean)
    sdv = pd.pivot_table(df, index=index_col, values=val_col, columns=hue_col, aggfunc=np.std)
    spivot.plot(ax=ax)
    ax.fill_between(np.arange(24), (spivot.min(1)-sdv.min(1)), 
                        (spivot.max(1)+sdv.max(1)),  color="lightsteelblue", alpha=0.5)
    return ax


In [ ]:
res_pt=create_res(pt_data.copy(),pt_netload.copy())
res_pt = prepareheatmap_df(res_pt)
#res_uk.head()

In [ ]:
az.style.use(["science",  "arviz-white", 'tableau-colorblind10'])
for model in ['MLPF', 'NBEATS',  'TimesNet']:
    fig, ax = plt.subplots(1, 1, figsize=(12, 3.0))
    hm=create_heatmap(ax, res_pt, value=model, index='Time', column='date', label=f'RES-{model}')
    fig.savefig(f'{figure_path}/PT_error_analysis_{model}.pdf', dpi=480)

In [ ]:
met_pt = prepareheatmap_df(metrics_pt)

In [ ]:
for model in ['MLPF', 'NBEATS',  'TimesNet']:
    fig, ax = plt.subplots(1, 1, figsize=(9, 2.0))
    
    hm=create_heatmap(ax, met_pt[met_pt.Model==model]['2021'], value='res', index='day', column='week', label=f'NRMSE-{model}', vmax=10, vmin=-10)
    fig.savefig(f'{figure_path}/PT_error_analysis_days_{model}.pdf', dpi=480)

In [ ]:
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=cats, ordered=True)

hm.index = hm.index.astype(cat_type)
hm.index

## Short long term trade-off

In [ ]:
from IPython.display import clear_output
import matplotlib.dates as mdates

experiment_type='short-long'
window_type='expanding'
exp_name='PT-Benchmark' 
dataset='pt_dataset'
encoder_type='MLPForecast'#'NHiTS',  'PatchTST', 'TimesNet'
short_long_metrics=[]
short_long_all_data={}
for horizon in [2, 12, 24, 48, 96, 192, 336]:
    data_per_window=[]
    for window_size in [48, 96, 144, 192, 336, 384]:#432
        file_name=f'{exp_name}_{dataset}_{experiment_type}_{window_size}_{horizon}_{window_type}'
        all_metrics=[]
    
        
        metrics_per_model=[]
        residual_per_model=[]
        data_per_model=[]
        print(window_size)
        print('')
        for cross in tqdm(range(1, 11)):
            result_path=f'../results/{file_name}/{encoder_type}/{cross}_cross_validation_processed_results.npy'
            results=np.load(result_path, allow_pickle=True).item()
            time_stamp=results['index']
            true=results['true']
            pred=results['pred']
            df = pd.DataFrame(time_stamp.flatten()[:len(true.flatten())])
            df.columns=['Date']
            df[f'{window_size}']=pred.flatten()
            df['true']=true.flatten()
            data_per_model.append(df)

            metrics=results['NetLoad_metrics']
            index=df.Date.dt.round("D").unique()


            metrics['Fold']=cross
            metrics['Model']=encoder_type
            #metrics['Train-time']=results['train-time'] if 'train-time' in results.keys() else results['train_time']
            metrics['Test-time']=results['test-time'] if 'test-time' in results.keys() else results['test_time']
            metrics=metrics.set_index('timestamp')
            metrics=add_time_features(metrics, hemisphere = 'Northern')
            metrics=metrics.replace('CATBOOST', 'CAT').replace('AutoARIMA', 'ARIMA').replace('LREGRESS', 'L-REG').replace('SeasonalNaive', 'S-Naive').replace('MLPForecast', 'MLPF')
            metrics_per_model.append(metrics)
            
        if window_size>=horizon:
            if horizon<48:
                fig, ax = plt.subplots(1,1, figsize=(3.8,2))
                hfmt = mdates.DateFormatter(' %H')
                ax.set_xlabel('Time ( Hour)')
                
            elif horizon==48:
                fig, ax = plt.subplots(1,1, figsize=(4,2))
                hfmt = mdates.DateFormatter('%H')
                ax.set_xlabel('Time ( Hour)')

            elif horizon>48 and horizon <=96:
                fig, ax = plt.subplots(1,1, figsize=(6,2))
                hfmt = mdates.DateFormatter('%d %H')
                ax.set_xlabel('Time (Date Hour)')

            elif horizon>96:
                fig, ax = plt.subplots(1,1, figsize=(9,2))
                hfmt = mdates.DateFormatter('%d %H')
                ax.set_xlabel('Time (Date Hour)')
            
            good=results['NetLoad_good']
            ax.plot(results['index'][good], results['true'][good][:, 0].flatten(),  ".", mec="#ff7f0e", mfc="None", label='True')
            ax.plot(results['index'][good], results['pred'][good][:, 0].flatten(),  color="#1f77b4", label="Pred")
            ax.legend()
            ax.set_yticklabels(np.arange(0, 100, 20))
            ax.set_ylim(0, 80)
            ax.set_ylabel('Power (kW)')
            
            ax.xaxis.set_major_formatter(hfmt)
            fig.savefig(f'{figure_path}/context_{window_size}_horizon_{horizon}_short_long.pdf', dpi=480)
            plt.close()
        
            
        metrics_per_model=pd.concat(metrics_per_model)
        all_metrics.append(metrics_per_model)
        data_per_model=pd.concat(data_per_model)
        data_per_window.append(data_per_model)
        metrics=pd.concat(all_metrics)
        metrics['Context']=window_size
        metrics['Horizon']=horizon
        

        if dataset=="UK":
            metrics['Dataset']='SPS-UK'
        else:
            metrics['Dataset']='MLVS-PT'
        
    
        del all_metrics
        del df 
        #del results
        del pred
        del true
        del data_per_model
        
        short_long_metrics.append(metrics)
        clear_output()
    short_long_all_data[horizon]=pd.concat(data_per_window)
short_long_metrics = pd.concat(short_long_metrics)

In [ ]:
res=short_long_metrics.groupby(['Context','Horizon'])[['nrmse', 'mae',  'nbias']].mean().round(2)
table = res.to_latex(index=True, formatters={"name": str.upper},
                  float_format="{:.2f}".format)
# Print the LaTeX table
print(table)

In [ ]:
res=short_long_metrics.groupby(['Model',    'Horizon'])[['nrmse', 'mae', 'nbias']].mean().round(2)
res

In [ ]:
def altair_plot(data, metric='mae', column='Context', dataset='PT',  across='Horizon', domain=[2., 6]):
    #data=data[data['Emb-Type']=='None']
    #data=data.replace('addition-comb', 'ADD').replace('attn-comb', 'ATTN').replace('weighted-comb', 'WTA')
    line = alt.Chart(data).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    y = alt.Y(f'mean({metric})', scale=alt.Scale(domain=domain), title=metric.upper()),
    x = alt.X(f'{column}:N', axis=alt.Axis( title=column.upper())),
    color=alt.Color('Season', scale=alt.Scale(scheme='tableau10')),
    column=alt.Column(f'{across}:N', title=across.upper()),
    
    ).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
        ).configure_view(
            strokeOpacity=0
        ).properties(width=120,
                        height=150
        )
    line.save(f'{figure_path}/{dataset}_{column}_{metric}.pdf', ppi=480)
    return line

In [ ]:
def altair_plot_overall(data, metric='mae', column='Context', dataset='PT',  across='Horizon', domain=[2., 6]):
    #data=data[data['Emb-Type']=='None']
    #data=data.replace('addition-comb', 'ADD').replace('attn-comb', 'ATTN').replace('weighted-comb', 'WTA')
    line = alt.Chart(data).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    y = alt.Y(f'mean({metric})', scale=alt.Scale(domain=domain), title=metric.upper()),
    x = alt.X(f'{column}:N', axis=alt.Axis( title=column.upper())),
    color=alt.Color(f'{across}:N', scale=alt.Scale(scheme='tableau10'), title=across.upper()),
    
    
    ).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
        ).configure_view(
            strokeOpacity=0
        ).properties(width=120,
                        height=150
        )
    line.save(f'{figure_path}/{dataset}_{column}_{metric}_overal.pdf', ppi=480)
    return line



In [ ]:
line=altair_plot_overall(short_long_metrics, domain=[2., 5])
line

In [ ]:
line=altair_plot(short_long_metrics, domain=[0., 6])
line

In [ ]:
line=altair_plot(short_long_metrics, across='Context', column='Horizon', domain=[0., 8])
line

In [ ]:
line=altair_plot_overall(short_long_metrics, across='Context', column='Horizon', domain=[2., 5])
line

In [ ]:
line=altair_plot(short_long_metrics, metric='nrmse', domain=[0.02, 0.1])
line

In [ ]:
line=altair_plot_overall(short_long_metrics, metric='nrmse', domain=[0.02, 0.1])
line

In [ ]:
line=altair_plot(short_long_metrics,  metric='nrmse', domain=[0.02, 0.1], across='Context', column='Horizon')
line

In [ ]:
line=altair_plot_overall(short_long_metrics,  metric='nrmse', domain=[0.02, 0.1], across='Context', column='Horizon')
line

In [ ]:
line=altair_plot(short_long_metrics,  metric='nbias', domain=[-5, 15], across='Context', column='Horizon')
line

In [ ]:
line=altair_plot_overall(short_long_metrics,  metric='nbias', domain=[-5, 15], across='Context', column='Horizon')
line

In [ ]:
line=altair_plot(short_long_metrics,  metric='nbias', domain=[-5, 15], column='Context', across='Horizon')
line

In [ ]:
line=altair_plot_overall(short_long_metrics,  metric='nbias', domain=[-5, 15], column='Context', across='Horizon')
line

### Statitsical test

In [ ]:
short_long_all_data[48]

In [ ]:

ax, lines, label=plot_prediction_with_upper_lower(ax,results['true'][good][:, 0].flatten(),
                                     results['pred'][good][:, 0].flatten(),
                                    None,
                                    None)